In [1]:
import os
import sys
import pandas
import numpy
import shutil
import cProfile, pstats, io

%load_ext cython

In [2]:
from package_parsers.sat_parser import SATParser
from algorithm_tester_common.tester_dataclasses import AlgTesterContext
from algorithm_tester.helpers import create_path
from algorithm_tester.concurrency_runners import BaseRunner

In [3]:
from package_algorithms.sat.sa_sat_v1 import SimulatedAnnealing_SAT_V1
from package_algorithms.sat.sa_sat_v2 import SimulatedAnnealing_SAT_V2
from package_algorithms.sat.sa_sat_v3 import SimulatedAnnealing_SAT_V3

In [4]:
def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

In [14]:
@profile
def profileRunAlgorithms(context, instance_data):
    sa = SimulatedAnnealing_SAT_V3()
    sa.perform_algorithm(context, instance_data)

parser: SATParser = SATParser()

out_dir: str = ".tmp_out_dir"
context: AlgTesterContext = AlgTesterContext(
    algorithms=["SA_SAT"], parser="SATParser", communicators=[], concurrency_runner=BaseRunner(),
    check_time=True, time_retries=1, max_num=None,
    extra_options={"init_temperature":1000, "min_temperature":1, "cooling":0.995, "cycles":50, "max_retry_attempts":3, "create_evo_file":False},
    input_dir="../data/NK", output_dir=out_dir
    )
    
create_path(out_dir)
with open("../data/wuf-A/wuf20-88-A/wuf20-01000-A.mwcnf") as instance_file:
    instance_data = parser.get_next_instance(instance_file)
    profileRunAlgorithms(context, instance_data)
    
shutil.rmtree(out_dir)

         5998249 function calls in 4.797 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    4.797    4.797 <ipython-input-14-8d212197a611>:1(profileRunAlgorithms)
        1    0.000    0.000    4.797    4.797 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithms/sat/sa_sat_v2.py:103(perform_algorithm)
        1    0.072    0.072    4.797    4.797 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithms/sat/sa_sat_v2.py:69(get_solution)
   206850    0.400    0.000    4.725    0.000 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithms/sat/sa_sat_v2.py:36(try_new_solution)
   206850    0.602    0.000    3.542    0.000 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithms/sat/sa_sat_v3.py:44(get_new_neighbour)
   206850    0.418    0.000    2.097    0.000 /Users/petr/Documents/Projects/Python/AlgorithmTester/package_algorithm

In [ ]:
%%cython

import time
import random
from libc.math cimport exp
from libc.stdlib cimport rand, srand, RAND_MAX
from libc.stdio cimport fopen, FILE, fclose, fprintf

cpdef void set_seed(int seed):
    srand(seed)
    
cpdef float random_float():
    return rand()/RAND_MAX

cpdef int random_int(int low = 0, int height = 2):
    return rand() % height + low
            
cpdef test(str path):
    with open(path, "w") as out:
        print(type(out))
        out.write("test\n")

In [ ]:
cooling = 0.995
max_temp = 1000
min_temp = 0.5
cycles = 50
cntr = 0

while max_temp > min_temp:
    for _ in range(cycles):
        cntr += 1
    max_temp *= cooling

cntr